In [1]:
from google import genai
from google.genai import types
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os
from io import BytesIO
import base64

client = genai.Client(
    api_key=os.environ.get('GEMINI_BILLING_ACCOUNT', None),
)

contents = ('''A hand-drawn, light watercolor illustration in a whimsical children's storybook style. The scene depicts a vertical cross-sectional (cutaway side view) of a slightly deeper-than-usual village stone well, revealing its full underground structure. The interior walls are made of aged, uneven bricks and worn stones, showing natural texture and history. The well extends deep underground, ending in a dry, rough floor with no water, emphasizing the buffalo’s predicament.

At the top of the well, a traditional wooden pulley is securely mounted between two rustic vertical posts. A thick rope hangs from the pulley, stretching down into the well’s depths. The pulley is simple, weathered, and entirely in tune with a rural Indian village setting.

At the bottom of the well, a dark grey buffalo named Bubbly stands, looking worried and helpless. The buffalo is slightly chubby, with small curved horns, big innocent eyes, and a gentle, unsure posture. It is illustrated with soft lines, rounded shapes, and light watercolor shading that adds warmth and emotion.

Peering in from above the well, Bittu, a young Indian boy, is shown bending forward, curiously and anxiously looking down at Bubbly. He is about 10 years old, with medium brown skin, short, messy black hair, and is dressed in a light blue half-sleeve shirt and brown shorts. His full body is visible in the frame, naturally posed, with a soft and expressive face that captures his concern.

The surrounding cutaway earth is softly layered with light brown and ochre tones, capturing the feel of natural soil. The background above ground includes a few distant village huts and sparse trees, lightly sketched in soft watercolor washes, keeping the mood peaceful and rustic.

The overall illustration style is warm and inviting, with gentle outlines, soft pastel colors, light watercolor textures, and a storybook charm that brings emotion and narrative to the scene.''')

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save('gemini-native-image.png')
    image.show()


I will generate a light watercolor illustration in a whimsical children's storybook style, depicting a vertical cross-section of a deep village stone well. The aged, uneven brick and stone interior will lead down to a dry, rough bottom where a worried, slightly chubby dark grey buffalo named Bubbly with small curved horns and big innocent eyes stands. Above, a traditional wooden pulley with a thick rope will be mounted on rustic posts. Peering down into the well will be a concerned young Indian boy named Bittu, around 10 years old with medium brown skin, short messy black hair, wearing a light blue shirt and brown shorts. The cutaway earth will be layered in soft browns and ochre, and the background will show distant village huts and sparse trees in light watercolor washes, creating a warm, inviting, and rustic scene.




In [3]:
from PIL import Image
from google import genai

client = genai.Client(
    api_key=os.environ.get('GEMINI_BILLING_ACCOUNT', None),
)

image = Image.open("gemini-native-image.png")
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[image, "What does this image depict?"],
)
print(response.text)


This image depicts a pile of **French fries** on a white plate.


In [11]:
import time
from google import genai
import os
from google.genai import types

client = genai.Client(
    api_key=os.environ.get('GEMINI_BILLING_ACCOUNT', None),
)

prompt = """Drone shot following a classic red convertible driven by a man along a winding coastal road at sunset, waves crashing against the rocks below.
The convertible accelerates fast and the engine roars loudly."""

operation = client.models.generate_videos(
    model="veo-3.0-generate-preview",
    prompt=prompt,
)

# Poll the operation status until the video is ready
while not operation.done:
    print("Waiting for video generation to complete...")
    time.sleep(10)
    operation = client.operations.get(operation)

# The correct way to access generated videos according to the official docs
# The response structure varies between Veo 2 and Veo 3
try:
    # For Veo 3 (your current model), the structure is different
    # Check if it's the new response format
    if 'generateVideoResponse' in operation.response:
        # Veo 3 format
        generated_samples = operation.response['generateVideoResponse']['generatedSamples']
        for n, sample in enumerate(generated_samples):
            video_uri = sample['video']['uri']
            
            # Download using authenticated request with API key
            import requests
            
            # Append API key to the URI for authentication
            api_key = os.environ.get('GEMINI_BILLING_ACCOUNT', None)
            if api_key:
                download_url = f"{video_uri}&key={api_key}"
                
                response = requests.get(download_url)
                if response.status_code == 200:
                    filename = f"realism_example_{n}.mp4"
                    with open(filename, "wb") as f:
                        f.write(response.content)
                    print(f"Generated video saved to {filename}")
                else:
                    print(f"Failed to download video {n}. Status code: {response.status_code}")
                    print(f"Response: {response.text}")
            else:
                print("API key not found. Cannot download video.")
                
    else:
        # Veo 2 format (from official docs)
        for n, generated_video in enumerate(operation.response.generated_videos):
            client.files.download(file=generated_video.video)
            generated_video.video.save(f"realism_example_{n}.mp4")
            print(f"Generated video saved to realism_example_{n}.mp4")

except Exception as e:
    print(f"Error: {e}")
    print("Response structure:", operation.response)
    
    # Fallback: try to manually extract and download
    try:
        if 'generateVideoResponse' in operation.response:
            generated_samples = operation.response['generateVideoResponse']['generatedSamples']
            for n, sample in enumerate(generated_samples):
                video_uri = sample['video']['uri']
                print(f"Video {n} URI: {video_uri}")
                print("To download manually, append your API key: &key=YOUR_API_KEY")
    except:
        print("Could not extract video URIs")

print("Video generation process completed!")

Waiting for video generation to complete...
Waiting for video generation to complete...
Waiting for video generation to complete...
Waiting for video generation to complete...
Waiting for video generation to complete...
Generated video saved to realism_example_0.mp4
Video generation process completed!
